## IMDb

At Fast.ai we have introduced a new module called fastai.text which replaces the torchtext library that was used in our 2018 dl1 course. The fastai.text module also supersedes the fastai.nlp library but retains many of the key functions.

In [2]:
from fastai.text import *
from fastai.core import num_cpus, partition_by_cores
import html
from pathlib import Path
import numpy as np
import csv
import pandas as pd
from collections import Counter, defaultdict
from itertools import chain
from nltk.corpus import brown
import os, re

from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from typing import Callable, List, Collection
from concurrent.futures.process import ProcessPoolExecutor

The Fastai.text module introduces several custom tokens.

We need to download the IMDB large movie reviews from this site: http://ai.stanford.edu/~amaas/data/sentiment/
Direct link : [Link](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) and untar it into the PATH location. We use pathlib which makes directory traveral a breeze.

In [3]:
PATH=Path('/media/discoD/repositorios/1-billion-word-language-modeling-benchmark/')

## Standardize format

The imdb dataset has 3 classes. positive, negative and unsupervised(sentiment is unknown). 
There are 75k training reviews(12.5k pos, 12.5k neg, 50k unsup)
There are 25k validation reviews(12.5k pos, 12.5k neg & no unsup)

Refer to the README file in the imdb corpus for further information about the dataset.

In [4]:
class VocabularyTokenizer():
    "Put together rules, a tokenizer function and a language to tokenize text with multiprocessing."
    def __init__(self, tok_func:Callable=SpacyTokenizer, lang:str='pt', n_cpus:int=None):
        self.tok_func,self.lang = tok_func,lang
        self.n_cpus = n_cpus or num_cpus()//2

    def process_text(self, t:str, tok:BaseTokenizer) -> List[str]:
        "Processe one text `t` with tokenizer `tok`."
        return tok.tokenizer(t)

    def _process_all_1(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts` in one process."
        tok = self.tok_func(self.lang)
        return [self.process_text(t, tok) for t in texts]

    def process_all(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts`."
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])

In [5]:
def save_texts(paths, filename, lang):
    CLASSES = ['unsup']
    file_count = 0
    filename = filename + '_' + lang + '.csv'
    if os.path.isfile(filename):
        os.remove(filename)
    with open(filename, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
        for idx,label in enumerate(CLASSES):
            for path in paths:
                for fname in (path).glob('*'):
                    file_count += 1
                    print('writing from %s' % fname)
                    [writer.writerow([line, idx]) for line in fname.open('r', encoding='utf-8').read().split('\n')]
    print('%d texts saved to %s' % (file_count, filename))

In [6]:
save_texts([PATH/'training-lener/'], 'train_lener', 'pt')
save_texts([PATH/'heldout-lener/'], 'test_lener', 'pt')
save_texts([PATH/'training-lener/',PATH/'heldout-lener/'], 'full_lener', 'pt')

writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00001-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00002-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00003-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00004-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00005-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00006-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00007-of-00010
writing from /media/discoD/repositorios/1-billion-word-language-modeling-benchmark/training-lener/lener-00008-of-00010
writing from /media/discoD/repositorios/1-billio

In [7]:
def get_tokens(filename):
    data = pd.read_csv(filename, header=None, escapechar='\\', chunksize=500000)
    for idx, df in enumerate(data):
        print(idx)
        yield VocabularyTokenizer().process_all(df[0].astype(str))

In [8]:
freq_full = Counter(p for o in chain.from_iterable(get_tokens('full_lener_pt.csv')) for p in o)
freq_full.most_common()

0


[('"', 20548),
 (',', 19925),
 ('de', 10790),
 ('.', 9966),
 ('a', 9196),
 ('o', 6713),
 ('do', 6296),
 ('da', 5423),
 ('que', 4558),
 ('e', 4238),
 ('/', 3400),
 ('em', 2804),
 (')', 2335),
 ('(', 2180),
 ('não', 2072),
 ('no', 1966),
 ('os', 1555),
 ('para', 1481),
 ('-', 1441),
 (':', 1420),
 ('à', 1386),
 ('por', 1269),
 ('com', 1244),
 ('se', 1233),
 ('na', 1129),
 ('dos', 1045),
 ('DE', 977),
 ('as', 877),
 (';', 850),
 ('art', 836),
 ('A', 827),
 ('ou', 817),
 ('pelo', 801),
 ('das', 739),
 ('como', 729),
 ('é', 726),
 ('pela', 706),
 ('Lei', 703),
 ('O', 616),
 ('ser', 592),
 ('nos', 586),
 ('d', 586),
 ('Tribunal', 583),
 ('§', 556),
 ('n', 543),
 ('nº', 525),
 ('Federal', 496),
 ('DO', 473),
 ('recurso', 448),
 ('foi', 446),
 ('decisão', 417),
 ('sua', 417),
 ('DA', 417),
 ('autos', 383),
 ('°', 367),
 ('uma', 363),
 ('artigo', 352),
 ('Justiça', 350),
 ('fls', 348),
 ('esta', 348),
 ('caso', 317),
 ("'", 316),
 ('sobre', 309),
 ('termos', 293),
 ('qual', 278),
 ('II', 278),


In [9]:
palavras = [palavra for palavra, contagem in freq_full.most_common()]
palavras

['"',
 ',',
 'de',
 '.',
 'a',
 'o',
 'do',
 'da',
 'que',
 'e',
 '/',
 'em',
 ')',
 '(',
 'não',
 'no',
 'os',
 'para',
 '-',
 ':',
 'à',
 'por',
 'com',
 'se',
 'na',
 'dos',
 'DE',
 'as',
 ';',
 'art',
 'A',
 'ou',
 'pelo',
 'das',
 'como',
 'é',
 'pela',
 'Lei',
 'O',
 'ser',
 'nos',
 'd',
 'Tribunal',
 '§',
 'n',
 'nº',
 'Federal',
 'DO',
 'recurso',
 'foi',
 'decisão',
 'sua',
 'DA',
 'autos',
 '°',
 'uma',
 'artigo',
 'Justiça',
 'fls',
 'esta',
 'caso',
 "'",
 'sobre',
 'termos',
 'qual',
 'II',
 'parte',
 'inciso',
 'Corte',
 'pagamento',
 'I',
 's',
 'Militar',
 'seu',
 'processo',
 'E',
 'quando',
 'sem',
 'acórdão',
 'um',
 'forma',
 'Ministro',
 'CLT',
 'qualquer',
 'Relator',
 'bem',
 'sentido',
 'este',
 'conforme',
 'há',
 'p.',
 'seja',
 'Constituição',
 'recorrente',
 'peça',
 'militar',
 'Súmula',
 'relação',
 'Acórdão',
 'fato',
 'ainda',
 'mesmo',
 'multa',
 'julgado',
 'razão',
 '2016',
 'Turma',
 'Superior',
 'deve',
 'nas',
 'apenas',
 'já',
 'revista',
 'Em',
 

In [10]:
len(palavras)

19175

In [11]:
palavras.insert(0, '<UNK>')
palavras.insert(0, '<S>')
palavras.insert(0, '</S>')
palavras

['</S>',
 '<S>',
 '<UNK>',
 '"',
 ',',
 'de',
 '.',
 'a',
 'o',
 'do',
 'da',
 'que',
 'e',
 '/',
 'em',
 ')',
 '(',
 'não',
 'no',
 'os',
 'para',
 '-',
 ':',
 'à',
 'por',
 'com',
 'se',
 'na',
 'dos',
 'DE',
 'as',
 ';',
 'art',
 'A',
 'ou',
 'pelo',
 'das',
 'como',
 'é',
 'pela',
 'Lei',
 'O',
 'ser',
 'nos',
 'd',
 'Tribunal',
 '§',
 'n',
 'nº',
 'Federal',
 'DO',
 'recurso',
 'foi',
 'decisão',
 'sua',
 'DA',
 'autos',
 '°',
 'uma',
 'artigo',
 'Justiça',
 'fls',
 'esta',
 'caso',
 "'",
 'sobre',
 'termos',
 'qual',
 'II',
 'parte',
 'inciso',
 'Corte',
 'pagamento',
 'I',
 's',
 'Militar',
 'seu',
 'processo',
 'E',
 'quando',
 'sem',
 'acórdão',
 'um',
 'forma',
 'Ministro',
 'CLT',
 'qualquer',
 'Relator',
 'bem',
 'sentido',
 'este',
 'conforme',
 'há',
 'p.',
 'seja',
 'Constituição',
 'recorrente',
 'peça',
 'militar',
 'Súmula',
 'relação',
 'Acórdão',
 'fato',
 'ainda',
 'mesmo',
 'multa',
 'julgado',
 'razão',
 '2016',
 'Turma',
 'Superior',
 'deve',
 'nas',
 'apenas',


In [12]:
sum(freq_full.values())

290025

In [13]:
def write_list(array, filename):
    with open(filename, 'w') as file:
        for item in array:
            file.write(item + '\n')
    file.close()

In [14]:
write_list(array=palavras, filename='vocabulario_lener.txt')

In [15]:
!tail -n 10 vocabulario_lener.txt

NESSA
Impetrado
106068
PUNITIVA
214-215v
FI
RR-180400-29.2004.5.03.0044
terminantemente
Arquivem-se
2015.Embargos


In [16]:
singletons = [palavra for palavra, contagem in freq_full.most_common() if contagem == 1]
print(len(singletons))
singletons

7783


['volitivo',
 'determino',
 'reautuação',
 'amenizam',
 'impeditivas',
 'VMF',
 'lm',
 'rs',
 'gastava',
 'cariz',
 'autor-recorrido',
 'desincumbir',
 'sobrelabor',
 'alvitrado',
 'confessa',
 'radicais',
 '24.11.2017',
 '14539902',
 'fixá-los',
 'Moacir',
 'VENCIMENTO',
 'PETIÇÃO',
 '227.',
 '623-2',
 '30371BA',
 'claras',
 'SUSTENTAÇÃO',
 'ORAL',
 'uniformizador',
 'disciplinamento',
 'voltado',
 'uniformizadores',
 'executados',
 'diligenciar',
 'periódico',
 'prestaram',
 'adotasse',
 'condeno',
 'zelando',
 'concedi',
 'migrar',
 'Adesão',
 'Regras',
 'Saldamento',
 'PLAN',
 'Novação',
 'Previdenciários',
 'transacionada',
 'formando',
 'DESPACHOS',
 'SECRETÁRIA',
 'tramitando',
 'desconstitui',
 'regulamenta',
 'Relatados',
 'acabamento',
 'caucionados',
 'NEGANDO',
 'dedicado',
 'desconstruir',
 'POLO',
 'PASSIVO',
 'Sobrevindo',
 'Responsabilização',
 'Qualificado',
 'narrada',
 'AGENTE',
 '61-93',
 'BIS',
 'IDEM',
 'Rompimento',
 'explicando',
 'cuidou-se',
 'gostaria',
 'con

In [17]:
currency_pattern = r"\b(?<![.,-])[0-9]{1,3}(?:,?[0-9]{3})*\.[0-9]{2}(?![.,-])\b|\b(?<![.,-])[0-9]{1,3}(?:.?[0-9]{3})*\,[0-9]{2}(?![.,-])\b"

In [18]:
valores = [singleton for singleton in singletons if re.match(pattern=currency_pattern, string=singleton)]
print(len(valores))
valores

36


['2.510.200,00',
 '2.379.599,00',
 '32,22',
 '9.020.221.885,00',
 '339457,71',
 '417,48',
 '240.000,00',
 '10.000,00',
 '326.264,29',
 '43,45',
 '1.863.107.652,00',
 '878.782,18',
 '22.141.243.230,00',
 '3.855.600,00',
 '20.000,00',
 '23.250,00',
 '1.214.778,68',
 '17.905.528.190,00',
 '2,98',
 '1,69',
 '21.923,70',
 '14.820,13',
 '2.263,96',
 '13.333,76',
 '5320,50',
 '79.807,50',
 '9.10',
 '26.925.750.075,00',
 '6.700,00',
 '244.743,00',
 '4.784.506.845,00',
 '1.072.028,13',
 '28.788.857.727,00',
 '33.300,00',
 '63.846,00',
 '85.128,00']